<a href="https://colab.research.google.com/github/raajmalhotra143/Android-apps/blob/main/Learning/DSA_by_AC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setup for running c++ codes**


In [ ]:
from IPython.core.magic import register_cell_magic
import subprocess
import os
import re # Import re for regex
import IPython # Import IPython module

@register_cell_magic
def cpp(line, cell):
    # Regex to find file blocks and extract filenames
    file_blocks_raw = re.split(r'^\s*--- FILE:\s*(\S+\.cpp)\s*---\s*$', cell, flags=re.MULTILINE)

    filenames_and_contents = {}
    main_filename_for_input_check = ""

    if len(file_blocks_raw) == 1:
        # No '--- FILE:' header found, treat the whole cell as a single C++ file.
        filenames_and_contents["main.cpp"] = cell.strip()
        main_filename_for_input_check = "main.cpp"
    elif not file_blocks_raw[0].strip(): # Starts with --- FILE:
        # The first element is empty, then filename, then content, then filename, etc.
        # [ '', 'file1.cpp', 'content1', 'file2.cpp', 'content2', ... ]
        for i in range(1, len(file_blocks_raw), 2):
            filename = file_blocks_raw[i].strip()
            content = file_blocks_raw[i+1].strip()
            filenames_and_contents[filename] = content
        main_filename_for_input_check = list(filenames_and_contents.keys())[-1] # Last file is main
    else:
        print("❌ Invalid C++ cell format. Ensure each file starts with '--- FILE: <filename.cpp> ---' or provide a single file without headers.")
        return

    if not filenames_and_contents:
        print("❌ No C++ code found in the cell or incorrect file block format.")
        return

    executable_name = "solution" # Fixed executable name for the final linked program

    object_files = []
    source_files = []

    # 1. Save C++ code to files and compile into object files
    print("--- COMPILING C++ FILES ---")
    for filename, cpp_code in filenames_and_contents.items():
        if filename.endswith(".cpp"):
            obj_filename = filename.replace(".cpp", ".o")
            source_files.append(filename)
            object_files.append(obj_filename)

            with open(filename, "w") as f:
                f.write(cpp_code)

            compile_process = subprocess.run(["g++", "-c", filename, "-o", obj_filename],
                                             capture_output=True, text=True)

            if compile_process.returncode != 0:
                print(f"❌ Compilation Failed for {filename}!")
                print(compile_process.stderr)
                # Clean up generated files before exiting
                for f in source_files:
                    if os.path.exists(f): os.remove(f)
                for f in object_files:
                    if os.path.exists(f): os.remove(f)
                return
            else:
                print(f"✅ Compilation Successful for {filename}!")
        else:
            print(f"Skipping non-C++ file: {filename}")

    # 2. Link all object files to create the executable
    print("\n--- LINKING OBJECT FILES ---")
    link_command = ["g++"] + object_files + ["-o", executable_name]
    link_process = subprocess.run(link_command, capture_output=True, text=True)

    if link_process.returncode != 0:
        print("❌ Linking Failed!")
        print(link_process.stderr)
        # Clean up generated files
        for f in source_files:
            if os.path.exists(f): os.remove(f)
        for f in object_files:
            if os.path.exists(f): os.remove(f)
        return
    else:
        print(f"✅ Linking Successful! Executable: {executable_name}")

    # 3. Handle input for the executable
    user_input = ""
    input_delimiter = "---INPUT---"
    interactive_input_needed = False

    # Check for specific input section in the *original full cell content*
    if input_delimiter in cell:
        parts = cell.split(input_delimiter, 1)
        # Input part is after the delimiter
        user_input = parts[1].strip()
    else:
        # Check if the designated 'main' file uses cin for interactive input
        if re.search(r'\bcin\s*>>', filenames_and_contents.get(main_filename_for_input_check, "")):
            print(f"Detected 'cin' in {main_filename_for_input_check}.")
            print("Please provide *all* required inputs for the C++ program below, one value per line.")
            print("To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.")
            print("Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.")

            collected_input_lines = []
            while True:
                try:
                    # Modified prompt for clarity
                    line_input = input("C++ Input (type empty line to finish)> ")
                    if not line_input:
                        break
                    collected_input_lines.append(line_input)
                except EOFError:
                    print("\nEOF received. Finishing input.")
                    break
            user_input = "\n".join(collected_input_lines)
            if user_input:
                interactive_input_needed = True

    # 4. Run the executable
    print("\n--- RUNNING PROGRAM ---")
    # If user_input was collected interactively or via ---INPUT---
    # We rely on the C++ program itself to echo the inputs if designed to.
    if user_input:
        # Do not print a summary of inputs here anymore,
        # as the C++ program is now expected to print its own prompts and echo values.
        pass
    else: # if no user_input was defined via ---INPUT--- or collected interactively
        # But still check if C++ code might implicitly ask for input (cin without ---INPUT---)
        if not re.search(r'\bcin\s*>>', filenames_and_contents.get(main_filename_for_input_check, "")):
            print("No input provided to C++ program.\n")

    run_process = subprocess.run([f"./{executable_name}"],
                                 input=user_input,
                                 capture_output=True,
                                 text=True)

    # 5. Display Output
    # The output from the C++ program already contains the prompts and echoed inputs.
    print(run_process.stdout)
    if run_process.stderr:
        print("Runtime Errors:", run_process.stderr)

    # 6. Clean up generated files
    print("\n--- CLEANING UP ---")
    if os.path.exists(executable_name):
        os.remove(executable_name)
        print(f"Removed executable: {executable_name}")
    for f in source_files:
        if os.path.exists(f):
            os.remove(f)
            print(f"Removed source file: {f}")
    for f in object_files:
        if os.path.exists(f):
            os.remove(f)
            print(f"Removed object file: {f}")

# Re-register the magic, in case it was already registered.
ip = IPython.get_ipython()
if ip and 'cpp' in ip.magics_manager.magics['cell']:
    del ip.magics_manager.magics['cell']['cpp']
if ip: # Only register if ip is not None
    register_cell_magic(cpp)


# **Lecture 2: Variables, Data Types & Operators | DSA Series (C++)**

This notebook covers every detail from the lecture, including memory concepts, variable rules, and operator logic.

---


setrup





## **1. Boilerplate Code (The Skeleton of a C++ Program)**

Every C++ program must follow a specific structure to work. This is called "Boilerplate Code."

* `#include <iostream>`: This is a **Preprocessor Directive**. It tells the compiler to include the `iostream` (Input-Output Stream) file. Without this, the computer does not know what `cout` (print) or `cin` (input) means.
* `using namespace std;`: The C++ standard library uses a namespace called `std`. Without this line, you would have to write `std::cout` every time. This line allows us to just write `cout`.
* `int main() { ... }`: This is the **Main Function**. The execution of every C++ program begins here. It returns an integer (`int`).
* `return 0;`: This signals that the program has finished running successfully. `0` usually means "no error".

In [29]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    // 'cout' stands for Console Output
    // 'endl' stands for End Line (moves cursor to next line)
    cout << "Apna College" << endl;

    // '\n' is another way to create a new line
    cout << "Hello World!" << "\n";

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

Apna College
Hello World!


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    // 'cout' stands for Console Output
    // 'endl' stands for End Line (moves cursor to next line)
    cout << "Apna College" << endl;

    // '\n' is another way to create a new line
    cout << "Hello World!" << "\n";

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

Apna College
Hello World!


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **2. Variables and Data Types**

A **Variable** is essentially a name given to a memory location. Think of it as a container or a box where we store data.

### **Rules for Naming Variables (Identifiers)**
1.  Must start with a letter (a-z, A-Z) or an underscore (`_`).
2.  Cannot start with a number (e.g., `1name` is invalid).
3.  No spaces allowed (e.g., `my age` is invalid; use `my_age` or `myAge`).
4.  C++ is **Case Sensitive** (`age`, `Age`, and `AGE` are three different variables).

### **Primitive Data Types**
| Data Type | Description | Size in Memory | Example |
| :--- | :--- | :--- | :--- |
| `int` | Integers (Whole numbers) | 4 Bytes (32 bits) | `int age = 25;` |
| `char` | Character (Single letter) | 1 Byte (8 bits) | `char grade = 'A';` |
| `float` | Floating Point (Small decimal) | 4 Bytes | `float pi = 3.14f;` |
| `bool` | Boolean (True or False) | 1 Byte | `bool isSafe = true;` |
| `double` | Double Precision (Large decimal) | 8 Bytes | `double price = 100.99;` |


In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int age = 25;
    char grade = 'A';
    float pi = 3.14f;
    bool isSafe = true;
    double price = 100.99;

    cout << "Age value: " << age << endl;
    cout << "Grade value: " << grade << endl;
    cout << "PI value: " << pi << endl;
    cout << "Is Safe (1=true, 0=false): " << isSafe << endl;
    cout << "Price value: " << price << endl;

    // Checking the size of an int in memory
    cout << "Size of int: " << sizeof(age) << " bytes" << endl;

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

Age value: 25
Grade value: A
PI value: 3.14
Is Safe (1=true, 0=false): 1
Price value: 100.99
Size of int: 4 bytes


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **3. Type Casting**

Type casting is the process of converting a variable from one data type to another.

1.  **Implicit Conversion:** Done automatically (Small -> Large type).
2.  **Explicit Conversion:** Forced by programmer (Large -> Small type).


In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    // 1. Implicit Conversion (Char -> Int)
    char grade = 'a'; // ASCII value of 'a' is 97
    int value = grade;
    cout << "ASCII value of 'a': " << value << endl;

    // 2. Explicit Conversion (Double -> Int)
    double bigNumber = 100.99;
    int smallNumber = (int)bigNumber; // Manually casting to int
    cout << "Original Double: " << bigNumber << endl;
    cout << "Casted Int (Decimals lost): " << smallNumber << endl;

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

ASCII value of 'a': 97
Original Double: 100.99
Casted Int (Decimals lost): 100


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **4. Input in C++ (Special Colab Instructions)**

**Issue:** You cannot type interactively in Colab while C++ code runs.
**Solution:** We create a text file (`input.txt`) and feed it to the program.

In [31]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int age;
    cout << "Enter your age: ";
    cin >> age; // Read age from input

    cout << "Your age is: " << age << endl;
    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 234
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Interactive input provided to C++ program (see 'C++ Input>' prompts above). Output below.
Enter your age: Your age is: 234


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **5. Operators & Problem Solving**

### **Problem 1: Sum of Two Numbers**

In [32]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter first number: ";
    cin >> a;
    cout << "Enter second number: ";
    cin >> b;

    int sum = a + b;
    cout << "Sum is: " << sum << endl;
    return 0;
}


--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 234
C++ Input (type empty line to finish)> 43
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Interactive input provided to C++ program (see 'C++ Input>' prompts above). Output below.
Enter first number: Enter second number: Sum is: 277


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


### **Problem 2: Unary Operator Logic (Pre vs Post Increment)**
* `a++`: Use then increment.
* `++a`: Increment then use.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int a = 10;
    int b = a++; // Post-increment: b gets 10, THEN a becomes 11

    cout << "After b = a++ :" << endl;
    cout << "b = " << b << " (Old value)" << endl;
    cout << "a = " << a << " (Updated value)" << endl;

    int x = 10;
    int y = ++x; // Pre-increment: x becomes 11 FIRST, then y gets 11

    cout << "After y = ++x :" << endl;
    cout << "y = " << y << " (Updated value)" << endl;
    cout << "x = " << x << " (Updated value)" << endl;

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

After b = a++ :
b = 10 (Old value)
a = 11 (Updated value)
After y = ++x :
y = 11 (Updated value)
x = 11 (Updated value)


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter first number: ";
    cin >> a;
    cout << "Enter second number: ";
    cin >> b;

    int sum = a + b;
    cout << "Sum is: " << sum << endl;
    return 0;
}
---INPUT---
5
10

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
Input provided via '---INPUT---' delimiter:
5
10

Enter first number: Enter second number: Sum is: 15


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **Homework: Calculator**
Performs +, -, *, / on two numbers.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    double num1, num2;
    cout << "Enter num1: ";
    cin >> num1;
    cout << "Enter num2: ";
    cin >> num2;

    cout << "Addition: " << (num1 + num2) << endl;
    cout << "Subtraction: " << (num1 - num2) << endl;
    cout << "Multiplication: " << (num1 * num2) << endl;
    cout << "Division: " << (num1 / num2) << endl;

    return 0;
}


--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 342
C++ Input (type empty line to finish)> 543
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Interactive input provided to C++ program (see 'C++ Input>' prompts above). Output below.
Enter num1: Enter num2: Addition: 885
Subtraction: -201
Multiplication: 185706
Division: 0.629834


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int age;
    cout << "Enter your age: ";
    cin >> age; // Read age from input

    cout << "Your age is: " << age << endl;
    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 324
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Interactive input provided to C++ program (see 'C++ Input>' prompts above). Output below.
Enter your age: Your age is: 324


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o
